# Passivation of Silicon Nanowire

## 0. Introduction.

In this notebook, we will demonstrate how to passivate a Silicon Nanowire using the Coordination Based Passivation Builder. The passivation is based on the coordination number of the atoms in the material. The passivation builder will saturate the bonds of the atoms with a coordination number below a certain threshold. The threshold can be set interactively or manually.
Following the publication:

> **B. Aradi, L. E. Ramos, P. Deák, Th. Köhler, F. Bechstedt, R. Q. Zhang, and Th. Frauenheim**,
> Theoretical study of the chemical gap tuning in silicon nanowires
> Phys. Rev. B 76, 035305 (2007)
> DOI: [10.1103/PhysRevB.76.035305](https://doi.org/10.1103/PhysRevB.76.035305)


This notebook will focus on recreating the material from Fig. 1. (Silicon nanowire passivated with hydrogen):

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/passivation/passivation_edge_nanowire_silicon/0-figure-from-manuscript.webp" alt="Silicon Nanowire Passivated with Hydrogen" width="400"/>

## 1. Prepare the environment
### 1.1. Set up the notebook

In [ ]:
# Enable interactive selection of coordination threshold
IS_COORDINATION_SELECTION_INTERACTIVE = False

BOND_LENGTH = 1.46  # in Angstroms
PASSIVANT = "H"  # Chemical symbol of the passivant
COORDINATION_SEARCH_RADIUS = 2.5  # in Angstroms (sphere in which to search for neighbors)
COORDINATION_THRESHOLD = 3  # Coordination number below which to passivate
MAX_BONDS_TO_SATURATE = 2  # Maximum number of bonds to saturate

 ### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

## 2. Create the Silicon Nanowire
### 2.1. Get input material
Import the Silicon from Standata

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

silicon = Material.create(Materials.get_by_name_first_match('Silicon'))

### 2.2. Create a silicon nanowire
Nanowire is cut from a slab of Silicon with a box filter, crystal is rotated at an angle to allow for precise cut.

In [ ]:
from mat3ra.made.tools.operations.core.unary import rotate
from mat3ra.made.tools.modify import filter_by_box, add_vacuum_sides, translate_to_center
from mat3ra.made.tools.helpers import create_slab

silicon_slab = create_slab(
    crystal=silicon, 
    miller_indices=(1, 1, 0), 
    number_of_layers=2, 
    vacuum=0, 
    use_orthogonal_c=True,
    xy_supercell_matrix=[[3, 0], [0, 2]]
)

angle = 50
silicon_slab = translate_to_center(silicon_slab)
rotated_slab = rotate(silicon_slab, "z", -angle)
silicon_nanowire = filter_by_box(rotated_slab, [0.25, 0.2, 0], [0.85, 0.87, 1])

# visualize_materials([{"material": rotated_slab},{"material": silicon_nanowire}, {"material": silicon_nanowire, "rotation": "-90x"}])
nanowire_xy_pbc = rotate(silicon_nanowire, "z", 90+angle)
nanowire_xy_pbc = add_vacuum_sides(nanowire_xy_pbc, 10.0, on_x=True, on_y=True)

### 2.3. Visualize the Material

In [ ]:
from utils.visualize import visualize_materials

visualize_materials([{"material": nanowire_xy_pbc}, {"material": nanowire_xy_pbc, "rotation": "-90x"}])

### 2.4. Compute the radial distribution function (RDF) of the Material

In [ ]:
from utils.plot import plot_rdf

plot_rdf(nanowire_xy_pbc, cutoff=10.0)

## 3. Create passivated material
### 3.1. Set the passivation parameters including the coordination threshold

In [ ]:
from utils.io import select_coordination_threshold
from mat3ra.made.tools.helpers import get_unique_coordination_numbers

coordination_numbers = get_unique_coordination_numbers(material = nanowire_xy_pbc,  cutoff=10.0)

print(f"Unique coordination numbers: {coordination_numbers}")

coordination_threshold = COORDINATION_THRESHOLD
if IS_COORDINATION_SELECTION_INTERACTIVE:
    coordination_threshold = await select_coordination_threshold(coordination_numbers, COORDINATION_THRESHOLD)

### 3.2. Create the passivated material

In [ ]:
from mat3ra.made.tools.helpers import passivate_dangling_bonds

passivated_nanowire = passivate_dangling_bonds(
    material=nanowire_xy_pbc,
    bond_length=BOND_LENGTH,
    passivant=PASSIVANT,
    shadowing_radius=COORDINATION_SEARCH_RADIUS,
    coordination_threshold=coordination_threshold,
    number_of_bonds_to_passivate=MAX_BONDS_TO_SATURATE,
)

## 4. Preview the Material

In [ ]:
visualize_materials(passivated_nanowire, viewer="wave")

## 5. Download the Material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

passivated_nanowire.name = "H-Passivated Silicon Nanowire"
set_materials(passivated_nanowire)
download_content_to_file(passivated_nanowire.to_json(), "passivated_nanowire.json")